In [5]:
from openai import OpenAI
from pydub import AudioSegment
from io import BytesIO
from openai import OpenAI
import asyncio

from openai import AsyncOpenAI
from openai.helpers import LocalAudioPlayer

client = OpenAI()
# 1. Generate short meditation transcript (~60 seconds)
def generate_transcript(prompt: str) -> str:
    response = client.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a meditation guide. Keep output under 100 words."},
            {"role": "user", "content": f"Create a 1-minute calming meditation for: {prompt}"},
        ],
    )
    return response['choices'][0]['message']['content'].strip()



openai = AsyncOpenAI()

async def text_to_speech(text: str) -> None:
    async with openai.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice="coral",
        input=text,
        instructions="Speak in calming tone to help the user meditate.",
        response_format="pcm",
    ) as response:
        await LocalAudioPlayer().play(response)

# 3. Mix with background
def mix_with_background(voice: BytesIO, bg_path: str = "Flowing_Focus.mp3") -> AudioSegment:
    speech = AudioSegment.from_file(voice, format="mp3")
    bg = AudioSegment.from_file(bg_path)

    bg = bg - 15  # reduce background volume
    bg = bg[:len(speech)] if len(bg) > len(speech) else bg * (len(speech) // len(bg) + 1)
    bg = bg[:len(speech)]  # match length exactly

    final = speech.overlay(bg)
    return final

# 4. Run pipeline
if __name__ == "__main__":
    prompt = "relaxation and breath awareness"
    print("Generating transcript...")
    transcript = generate_transcript(prompt)
    print("Transcript:\n", transcript)

    print("Generating speech...")
    speech_audio = text_to_speech(transcript)

    print("Mixing with background...")
    final_audio = mix_with_background(speech_audio, "background.mp3")

    print("Exporting...")
    final_audio.export("final_meditation.mp3", format="mp3")
    print("✅ Done: saved as final_meditation.mp3")


Generating transcript...


AttributeError: 'OpenAI' object has no attribute 'ChatCompletion'

In [ ]:
import asyncio

from openai import AsyncOpenAI
from openai.helpers import LocalAudioPlayer

# load dotenv
from dotenv import load_dotenv
load_dotenv()

openai = AsyncOpenAI()

async def main() -> None:
    async with openai.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice="coral",
        input="Today is a wonderful day to build something people love!",
        instructions="Speak in a cheerful and positive tone.",
        response_format="pcm",
    ) as response:
        await LocalAudioPlayer().play(response)

if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop